# ONEWAY EXTRACTION

In [1]:
import numpy as np

import pandas as pd # To structure and manipulated data in a DataFrame format
import geopandas as gpd # To work with spatial data in a DataFrame
from geopandas import GeoDataFrame # To create a GeoDataFrame from a DataFrame

import shapely
from shapely.geometry import shape, Point, LineString # To create line geometries that can be used in a GeoDataFrame

from geographiclib.geodesic import Geodesic

#Thư viện quy đổi tọa độ -> met
from haversine import haversine, Unit
import matplotlib.pyplot as plt
plt.rcParams['axes.xmargin'] = 0.1
plt.rcParams['axes.ymargin'] = 0.1
%matplotlib inline


# INPUT DATA

In [2]:
day26 = 'data_pre_day26.csv'
day27 = 'data_pre.csv'
day_old = 'improved_preprocessing.csv'
day = day27
df= pd.read_csv(f'input/{day}')
df_lines = pd.read_csv('input/multipoint_new.csv')

### Add column: trip 

In [3]:
def add_trip_number(df):
    arr = df.loc[:, 'level_1'].values
    indices = np.where(arr == 0)[0]
    indices = np.append(indices, [len(df)])
    max_size = np.amax(np.diff(indices))
    duplicate_idx = np.repeat(indices, 2)[1:-1].reshape(-1, 2)

    def add_number(x):
        idx = np.where(indices == x[0])[0]
        result = np.full((x[1] - x[0],), idx)
        result = np.pad(result, (0, max_size - len(result)), 'constant', constant_values=-1)
        return result

    trip_number_result = np.apply_along_axis(add_number, 1, duplicate_idx)
    trip_number_result = trip_number_result[trip_number_result != -1]
    df['trip'] = trip_number_result
    return df

In [4]:
def add_heading(df):
    def calculate_heading(df):
        heading_list = []
        for i in df.index.tolist()[:-1]:
            heading = Geodesic.WGS84.Inverse(df.y[i], df.x[i], df.y[i + 1], df.x[i + 1])['azi1'] % 360
            heading_list.append(heading)
        heading_list.append(heading)
        return heading_list
    
    heading_list = []
    for i in df.trip.unique():
        heading_list.extend(calculate_heading(df.loc[df.trip == i]))
    
    df['heading'] = heading_list
    return df

In [5]:
# rename x, y cols
df.rename({'x':'y', 'y':'x'}, axis=1, inplace=True)
# rename label col
df_lines.rename({'label': 'trip'}, axis=1, inplace=True)

add_trip_number(df)
add_heading(df_lines)

,x,y,trip,heading
0,106.674185,10.765471,0,359.027196
1,106.674183,10.765641,0,340.364233
2,106.674107,10.765849,0,352.649092
3,106.674071,10.766128,0,334.215577
4,106.673958,10.766359,0,334.215577
...,...,...,...,...
3138,106.693337,10.792257,224,76.953618
3139,106.693522,10.792299,224,72.353057
3140,106.693708,10.792358,224,89.943567
3141,106.693933,10.792358,224,80.011852


In [6]:
df.describe()

,level_1,speed,x,y,heading,vehicletype,time_interval,distance,heading_interval,trip
count,947473.000000,947473.000000,947473.000000,947473.000000,947473.000000,947473.000000,947473.000000,947473.000000,947473.000000,947473.000000
mean,3.111588,16.801576,106.685188,10.777796,174.259182,361.999867,113.206433,76.492176,16.266321,93036.764784
std,4.458286,8.809780,0.008680,0.007615,99.537086,231.662415,1311.552002,60.722389,43.114676,54573.834883
min,0.000000,1.055640,106.664300,10.765434,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,10.000000,106.679855,10.770985,76.722228,300.000000,10.000000,35.198843,1.576490,44266.000000
50%,2.000000,16.000000,106.686874,10.777709,153.379165,300.000000,15.000000,62.471156,4.213520,93552.000000
75%,4.000000,23.000000,106.692160,10.784193,253.948933,700.000000,30.000000,101.115673,11.463982,140835.000000
max,209.000000,89.000000,106.698080,10.792480,359.759276,900.000000,82247.000000,1139.572105,359.093680,187110.000000


### Format DF

In [7]:
# geometry = [Point(xy) for xy in zip(df.x, df.y)]
# # gdf = df.drop(['y', 'x'], axis=1)
# # gdf = gdf.sort_values('datetime', ascending = True)
# gdf = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

# drop columns
df.drop(['level_1','speed','vehicletype', 'time_interval','distance'], axis=1, inplace=True)

In [8]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(['vehicle', 'datetime'], ascending=[True, True])
df.reset_index(drop=True, inplace=True)
df['heading_interval'] = 0

## Test data

In [9]:
df_test = df.copy()

In [10]:
df_test.head()

,vehicle,datetime,x,y,heading,heading_interval,trip
0,15A24926,2018-04-27 08:59:20,106.667780,10.765512,347.343620,0,0
1,15A24926,2018-04-27 08:59:30,106.667656,10.766058,346.031122,0,0
2,15A24926,2018-04-27 08:59:40,106.667496,10.766694,347.932335,0,0
3,15A24926,2018-04-27 08:59:50,106.667400,10.767138,347.932335,0,0
4,15A24926,2018-04-27 09:01:10,106.667240,10.767723,336.526522,0,1


# EXTRACTING ONEWAY ATTRIBUTE BY HEADING

<img src="img/oneway_extraction_alg.png" />

Source: <a href="https://drive.google.com/file/d/1gJY6XpEbzMk6-yuhgRB8D9Vd7anJq4Pf/view">
    Automatically Deriving and Updating Attribute Road Data from Movement Trajectories (chapter 5)
    <a/>

## helper functions: get trajec's center point & find nearest points  

KDTREE

In [11]:
# k is #dimensions
k = 2
def build_kd_tree(points, depth=0):
    axis = depth%k
    # choose root
    size = len(points)
    if size <= 0:
        return None
    
    sorted_points = sorted(points, key=lambda p: p[axis])
    root = size//2
    return {
        'point': sorted_points[root],
        'left': build_kd_tree(sorted_points[:root], depth+1),
        'right': build_kd_tree(sorted_points[root+1:], depth+1)
    }

def find_knearest_point(kdtree, depth=0, target=(0,0), knearest=set(), eps=10):
    # empty tree
    if kdtree == None:
        return knearest
    
    near_target = None
    next_branch = None
    root = kdtree['point']
    droot = haversine(target, root[0:2], unit=Unit.METERS)

    if droot <= eps:
        near_target = root
        knearest.add(near_target)

    # choose turn left or right
    axis = depth%k
    if target[axis] <= root[axis]:
        next_branch = 'left'
        opposite_branch = 'right'
    else:
        next_branch = 'right'
        opposite_branch = 'left'
    knearest = find_knearest_point(kdtree[next_branch], depth+1, target, knearest, eps)
    
    # check if opposite branch has chance store nearest point
    next_axis = (axis+1)%k
    if kdtree[opposite_branch] == None:
        return knearest
    opposite_root = kdtree[opposite_branch]['point']
    projection_point = None
    # get projection from target to opposite bounding box
    if next_axis == 0:
        projection_point = (opposite_root[next_axis], target[axis])
    if next_axis == 1:
        projection_point = (target[axis], opposite_root[next_axis])
    # recursive if bounding box is near target
    if haversine(projection_point, target, unit=Unit.METERS) <= eps:
        knearest = find_knearest_point(kdtree[opposite_branch], depth+1, target, knearest, eps)
    
    return knearest

Get trajecs center point

In [12]:
''' get_center_point() params:
input:
    gdf_single_trajec: geodataframe - a single trajectory
output:
    center_point: geodataframe - a single point
'''
def get_center_point(gdf_single_trajec):
    trajec_len = len(gdf_single_trajec)
    center = int(trajec_len/2)
    # get center point gdf
    center_point = gdf_single_trajec.iloc[center]
    
    return center_point

Find all points around center point

In [13]:
''' find_nearest_points() params:
input:
    center_point: geodataframe - a single point
    df: dataframe - graph data
    eps: meter - radius (50m as default)
output:
    gdf_cluster: geodataframe - a group of point nearest center point
'''
def find_nearest_points(center_point, df, kdtree, eps=10):
    # extract cluster labeled
    knearest = set()
    knearest = find_knearest_point(kdtree, depth=0, target=(center_point.x, center_point.y), knearest=knearest, eps=eps)
    knearest = [i[2] for i in knearest]
    df_cluster = df.iloc[knearest].copy()
    
    return df_cluster



 


Add column: Heading interval

In [14]:
''' add_heading_interval() params:
input:
    df_cluster: dataframe - a group of point nearest center point
    center_point: geodataframe - a single point
output:
    df_cluster: dataframe - add heading_interval column
'''
def add_heading_interval(df_cluster, center_point):
    df_cluster['heading_interval'] = df_cluster.heading.apply(lambda heading: abs(center_point.heading-heading) % 360 ) 
    return df_cluster

## Attribute Extraction of "oneway" implement

In [15]:
''' detect_oneway() params:
input:
    gdf: geodataframe - all data
    trip_id: geodataframe - a sub gdf represent for a trip
    trips: list trip id
output:
    tag = 1, tag = -1 -> oneway
    tag = 0 -> no
    tag = -2 -> can't detect
'''
def detect_oneway(gdf, gdf_lines, kdtree, trip_id):
    tag = -2 # tag for
    # select trip from trip_id
    gdf_trip = gdf_lines[gdf_lines.trip == trip_id]    
    # SELECT RELATIVE DIRECTION FOR CLUSTER
    center_point = get_center_point(gdf_trip) # get mid point of a trip
    df_cluster = find_nearest_points(center_point, gdf, kdtree, eps=10)
    if df_cluster.empty:
        return tag, df_cluster     # return if cluster is empty
    df_cluster.drop_duplicates(subset=['trip'], inplace=True) # drop trip duplicates
    
    cluster_size = len(df_cluster)
    # ignore cluster if the size is less than 10 points
    size_thresh = 5
    if cluster_size >= size_thresh:
        print(f'> {size_thresh}')
        # add heading_interval
        add_heading_interval(df_cluster, center_point)
        print(f'heading {center_point.heading}')
        # 
        radian_thresh = 30
        similar = len(df_cluster.loc[(df_cluster.heading_interval <= radian_thresh) | (df_cluster.heading_interval >= (360-radian_thresh))])
        opposite = len(df_cluster.loc[(df_cluster.heading_interval >= (180-radian_thresh)) & (df_cluster.heading_interval <= (180+radian_thresh))])
        total = len(df_cluster)
        print(f'similar: {similar}')
        print(f'opposite: {opposite}')
        print(f'total: {total}')

        # tag = 1, tag = -1 -> oneway
        # tag = 0 -> no
        rate_thresh = 0.9
        if similar == 0 and opposite == 0:
            pass
        elif similar/total > rate_thresh:
            tag = 1
        elif opposite/total > rate_thresh:
            tag = -1
        else:
            tag = 0
    else:
        print(f'< {size_thresh}')
    return tag, df_cluster

## ONEWAY EXTRACTION

In [16]:
# add oneway col
df_test['oneway'] = -2
# get list trajecs
trips = list(df_lines.trip.unique())
len(trips)

225

In [17]:
%%time
pts = list(zip(df_test.x, df_test.y))
pts = [(pts[i][0], pts[i][1], i) for i in range(len(pts))]
kdtree = build_kd_tree(pts)       

Wall time: 11.1 s


In [18]:
%%time
while trips != []:
    trip_id = trips.pop()
    print(f'in:trip len= {len(trips)} ====== trip id = {trip_id}')
    # run
    oneway_tag, cluster_trips = detect_oneway(df_test, df_lines, kdtree, trip_id)
    # add tag into data
    df_lines.loc[df_lines.trip == trip_id,'oneway'] = oneway_tag
    print('======')

in:trip len= 224 ====== trip id = 224
> 5
heading 89.94356650370548
similar: 39
opposite: 46
total: 90
in:trip len= 223 ====== trip id = 223
> 5
heading 292.782216495616
similar: 163
opposite: 167
total: 332
in:trip len= 222 ====== trip id = 222
> 5
heading 31.753667275292134
similar: 0
opposite: 2
total: 476
in:trip len= 221 ====== trip id = 221
> 5
heading 68.82447776189738
similar: 12
opposite: 16
total: 28
in:trip len= 220 ====== trip id = 220
> 5
heading 280.90238993149165
similar: 3
opposite: 10
total: 13
in:trip len= 219 ====== trip id = 219
> 5
heading 273.28951036506464
similar: 6
opposite: 5
total: 11
in:trip len= 218 ====== trip id = 218
> 5
heading 88.66683393285699
similar: 0
opposite: 18
total: 18
in:trip len= 217 ====== trip id = 217
> 5
heading 84.58367815160757
similar: 0
opposite: 41
total: 41
in:trip len= 216 ====== trip id = 216
> 5
heading 64.71065700545523
similar: 20
opposite: 11
total: 31
in:trip len= 215 ====== trip id = 215
> 5
heading 298.5212009521989
simila

## VISUALIZING DATA

In [19]:
import folium 

### helper functions: visualize into folium map

In [20]:
colors = [
    'green',
    'red',
    'orange',
    'yellow',
    'pink']

def create_map_obj():
    #Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 800)
#     mapobj = folium.Map([10.778015126603638, 106.68162304214593], zoom_start = 15, tiles='Cartodb dark_matter')
    mapobj = folium.Map([10.778015126603638, 106.68162304214593], zoom_start = 15)
    mapobj.add_to(f)
    return mapobj

def add_point(mapobj, df, colors):
    #Nạp x,y từ dataframe vào list coords
    coords = list(zip(df.y, df.x))
    #Hiển thị trên mapobj
    for coord in coords:
        folium.CircleMarker(location = coord,
                            radius = 2, 
                            fill = True,
                            fill_opacity = 1,
                            color = colors,
                            weight = 0.05).add_to(mapobj)
           
def add_lines(mapobj, df, color):
    coords = list(zip(df.y, df.x))
    folium.PolyLine(coords, color=color, weight=2, opacity=1).add_to(mapobj)
       
    
'''
Hàm hiển thị map.
Tùy chọn:
- mapobj: bản đồ nền
- gdf: geodataframe
- start: lộ trình bắt đầu
- end: lộ trình kết thúc
- mask_type: dạng đường (1) và dạng điểm (0)
'''
def show_n_route(mapobj, gdf, mask_type, single_color=-1):
    # Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 600)
    mapobj.add_to(f)

    # Get list trajecs id
    trajecs_id = list(gdf.trip.unique())
    # Show every single trajec
    for idx, id in enumerate(trajecs_id):
        # get sub trajec
        subgdf = gdf[gdf['trip'] == id]
        color_key = idx
        # coloring with only 1 color
        if single_color != -1:
            color_key = single_color
        # coloring into mapobj
        if mask_type == 0:
            add_point(mapobj, subgdf, colors[color_key % len(colors)])
        else:
            add_lines(mapobj, subgdf, colors[color_key % len(colors)])
  
    return mapobj

def show_n_route_label(mapobj, df, mask_type):
    # Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 600)
    mapobj.add_to(f)

    # Get list label
    labels = list(df.label.unique())
    # Show every single trajec
    for idx, label in enumerate(labels):
        # get sub trajec
        subdf = df[df['label'] == label]
        # color key
        if mask_type == 0:
            if label == (len(colors)-1):
                label = 0
            add_point(mapobj, subdf, colors[ label % len(colors)])
        else:
            add_lines(mapobj, subdf, colors[ label % len(colors)])
  
    return mapobj

def show_n_route_oneway(mapobj, gdf):
    # Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 600)
    mapobj.add_to(f)

    # Get list trajecs id
    trajecs_id = list(gdf.trip.unique())
    # Show every single trajec
    for idx, id in enumerate(trajecs_id):
        # get sub trajec
        subgdf = gdf[gdf['trip'] == id]
        # color key
        oneway_tag = subgdf.iloc[0].oneway
        if oneway_tag == -1:
            oneway_tag = 1
        '''
        tag = 1:  same direction with trip_id
        tag = -1: opposite direction
        tag = 0: at least 2 way
        '''
        add_point(mapobj, subgdf, colors[int(oneway_tag) % len(colors)])

    return mapobj

### Show lines

In [21]:
# map
mapobj1 = create_map_obj()
# show lines
show_n_route(mapobj1,df_lines, 1,single_color=1)
# # show points fisrt
# show_n_route(mapobj1, df_lines, 0, single_color=0)


### Show oneway points

In [22]:
# map
mapobj_oneway = create_map_obj()
'''
tag = 1 (red):  same direction with trip_id
tag = -1 (red): opposite direction
tag = 0 (yellow): at least 2 way
tag = -2 (green): undefine
'''
show_n_route(mapobj_oneway,df_lines, 1, single_color=2)
show_n_route_oneway(mapobj_oneway, df_lines)


Saving output

In [23]:
fname1 = "output/network.html"
mapobj1.save(fname1)

In [24]:
fname_oneway = f"output/oneway_code.html"
mapobj_oneway.save(fname_oneway)

In [25]:
df_lines.to_csv('output/oneway.csv')